In [ ]:
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from sklearn.linear_model import LinearRegression
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
import yt.wrapper as yt
from collections import Counter
import json
import typing as tp
import ast
from datetime import datetime
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import operator
import re
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

In [ ]:
timezone_dict = {'UTC+10': 'Asia/Vladivostok',
 'UTC+11': 'Asia/Sakhalin',
 'UTC+12': 'Asia/Kamchatka',
 'UTC+2':  'Europe/Kaliningrad',
 'UTC+3':  'Europe/Moscow',
 'UTC+4':  'Europe/Ulyanovsk',
 'UTC+5':  'Asia/Yekaterinburg',
 'UTC+6':  'Asia/Omsk',
 'UTC+7':  'Asia/Krasnoyarsk',
 'UTC+8':  'Asia/Irkutsk',
 'UTC+9':  'Asia/Yakutsk'}

## КЦ -> CRM

In [ ]:
from_path = "//home/cloud_analytics/call_center/output/streaming_output"

drop_list_path =\
'//home/cloud_analytics/lunin-dv/call_center_output_streaming_output/drop_task_id'

In [ ]:
df = lib.execute_query(f"""
SELECT
    task_id,
    'New' as Status,
    relevant_contact_name as Last_name,
    phone_1 as Phone_1,
    phone_2 as Phone_2,
    email as Email,
    last_communication_date as Last_communication_date,
    Lead_Source,
    'admin' as Assigned_to,
    Lead_Source_Description,
    time_to_recall as Callback_date,
    timezone as Timezone,
    name_contact as Account_name,
    'company name: ' || ifNull(name_contact, '')  
    || '; site: '    || ifNull(site, '') 
    || '; inn: '     || ifNull(inn, '')
    || '; location: '|| ifNull(location, '') as Description,
    'Комментарий от КЦ: ' || ifNull(comment, '') as Notes
FROM (
    SELECT
        argMax(timezone, assignment_approve_time) as timezone,
        argMax(time_to_recall, assignment_approve_time) as time_to_recall,
        argMax(relevant_contact_phone, assignment_approve_time) as phone_1,
        argMax(phone, assignment_approve_time) as phone_2,
        ifNull(argMax(lead_source_description, 
                        assignment_approve_time), 'Cold Calling') as Lead_Source_Description,
        ifNull(argMax(lead_source, assignment_approve_time), 'cold_calls') as Lead_Source,
        argMax(relevant_contact_name, assignment_approve_time) as relevant_contact_name,
        argMax(relevant_contact_email, assignment_approve_time) as email,
        toDateTime(MAX(assignment_approve_time/1000)) as last_communication_date,
        argMax(name_contact, assignment_approve_time) as name_contact,
        argMax(website, assignment_approve_time) as site,
        argMax(location, assignment_approve_time) as location,
        argMax(inn, assignment_approve_time) as inn,
        task_id,
        argMax(comment, assignment_approve_time) as comment
    FROM "//home/cloud_analytics/call_center/output/output"
    where toDate(assignment_approve_time/1000) >= toDate('2020-08-11')
    and status == 'success'
    and result == 'accept'
    GROUP BY task_id
) as call_center_info
FORMAT TabSeparatedWithNames
""".encode('utf-8'))

In [ ]:
df.drop_duplicates(inplace=True)
df['Email'] = df['Email'].apply(lambda x: '["' + x + '"]' if not pd.isnull(x) else np.nan)
df['Phone_1'] = df['Phone_1'].astype(str)
df['Phone_2'] = df['Phone_2'].astype(str)
df['Last_communication_date'] = df['Last_communication_date'].astype(str)
df['Email'] = df['Email'].astype(str)
df["Timezone"] = df["Timezone"].apply(lambda x: timezone_dict[x] if not pd.isnull(x) else 'Europe/Moscow')
df.replace('nan', np.nan, inplace=True)

In [ ]:
df['task_id'] = df.apply(lambda row: row['task_id'] if row['Last_communication_date'] < '2020-09-10 00:00:00'
                         else row['task_id'] + '_spark_id', axis=1)

Удалим старых

In [ ]:
drop_req = f"""
SELECT task_id from "{drop_list_path}"
FORMAT TabSeparatedWithNames
"""
drop_df = lib.execute_query(drop_req)
df = df[~df['task_id'].isin(drop_df['task_id'])]

In [ ]:
drop_df.shape

In [ ]:
assert len(df) < 11, f'Script wants to add {len(df)} leads'

In [ ]:
timestamp_req = """
SELECT
    toInt64(NOW()) as Timestamp
FORMAT TabSeparatedWithNames
"""
timestamp = lib.execute_query(timestamp_req)

In [ ]:
df['Timestamp'] = timestamp.iloc[0, 0]

In [ ]:
lib.save_table("update_leads", 
               "//home/cloud_analytics/export/crm/update_call_center_leads", df.drop(columns = ['task_id']),
               append=True)

## dropped list update

In [ ]:
new_drop_list_df = pd.DataFrame(df['task_id'])

In [ ]:
drop_table_name = drop_list_path.split('/')[-1]
drop_folder = drop_list_path[:-len(drop_table_name) - 1]

In [ ]:
lib.save_table(drop_table_name, drop_folder, new_drop_list_df, append=True)

In [ ]:
# drop_leads = lib.execute_query("""
# SELECT
#     DISTINCT 
#     toDateTime(date_entered),
#     toDateTime(1600093930),
#     dateDiff('minute', toDateTime(date_entered), toDateTime(1600093930)) as delta,
#     lead_source_description,
#     lead_id as CRM_Lead_ID
# FROM "//home/cloud_analytics/kulaga/leads_cube"
# WHERE dateDiff('minute', toDateTime(1600093930), toDateTime(date_entered)) <= 40
# AND dateDiff('minute', toDateTime(1600093930), toDateTime(date_entered)) >= 0
# FORMAT TabSeparatedWithNames
# """)
# drop_leads['Status'] = 'Recycled'
# drop_leads = drop_leads[['CRM_Lead_ID', 'Status']]

In [ ]:
# timestamp_req = """
# SELECT
#     toInt64(NOW()) as Timestamp
# FORMAT TabSeparatedWithNames
# """
# timestamp = lib.execute_query(timestamp_req).iloc[0, 0]

In [ ]:
# drop_leads['Timestamp'] = timestamp

In [ ]:
# drop_leads

In [ ]:
# lib.save_table("update_leads", 
#                "//home/cloud_analytics/export/crm/update_call_center_leads", drop_leads,
#                append=True)